In [ ]:
import os
import glob
import json
from dataclasses import dataclass
from typing import List, Dict, Any

import numpy as np
from openai import OpenAI

In [2]:
# Uses your OPENAI_API_KEY env var
client_oa = OpenAI()

In [3]:
@dataclass
class Chunk:
    id: int
    file_path: str
    chunk_index: int
    text: str
    score: float = 0.0 

In [4]:
from openai import OpenAI
from qdrant_client.models import PointStruct

EMBEDDING_MODEL = "text-embedding-3-small"

def get_embedding(text: str) -> list[float]:
    resp = client_oa.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text,
    )
    return resp.data[0].embedding

query = (
    "Generate telemetry configuration for Cisco IOS XR about BGP. "
    "Use gRPC with no TLS, telemetry server 192.0.2.0 port 57500. "
    "Choose relevant BGP sensor paths."
)
query_vector = get_embedding(query)


In [5]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

In [6]:
from qdrant_client.models import Filter, FieldCondition, MatchValue  # ← Your existing filter imports (unchanged)


# First search (with filter) — no Query/QueryVector wrapper needed for simple vector queries
hits = client.query_points(
    collection_name="catalog_embeddings",
    query=query_vector,                          # ← Pass vector directly (list[float])
    limit=10,
    with_payload=True,                        # ← Equivalent to with_payload=True (default)
).points  # ← Extract the list of points (same as old hits)


# Your print loop unchanged!
for h in hits[:5]:
    print(h.id, h.payload["yang_id"], h.payload["path"])

5 5 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data
4 4 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas
0 0 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma
3 3 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database
1 1 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/gsp-node-db-summary


In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue
from typing import List, Any, Dict, Optional

def retrieve_chunks(
    query: str,
    collection_name: str,
    top_k: int = 8,
    query_filter: Optional[Filter] = None,           # full flexibility
    score_threshold: Optional[float] = None         # optional minimum score
)-> List[Chunk]:
    """
    Universal retrieval function — works with ANY collection.
    
    Args:
        query: Natural language query
        collection_name: "yang_sensors", "yang_modules_chunks", etc.
        top_k: How many results to return
        query_filter: Optional Qdrant Filter (e.g. protocol_tag)
        score_threshold: Only return hits above this score
    
    Returns:
        List[Chunk] — same dataclass you already use
    """
    query_vector = get_embedding(query)

    hits = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        query_filter=query_filter,
        limit=top_k,                    # fetch more, then filter by score if needed
        with_payload=True,
    ).points

    results = []
    for hit in hits:
        if score_threshold is not None and hit.score < score_threshold:
            continue

        payload = hit.payload or {}

        # Robust fallback: different collections store text in different keys
        full_text = (
            payload.get("text") or
            payload.get("text_preview") or
            payload.get("description", "") + "\n" + payload.get("path", "")
        )

        chunk = Chunk(
            id=hit.id,
            file_path=payload.get("file_path") or payload.get("module", "unknown"),
            chunk_index=payload.get("chunk_index", 0),
            text=full_text
        )
        # Attach score for debugging/comparison
        chunk.score = hit.score  # type: ignore  (we add it dynamically)
        results.append(chunk)

    # Respect final top_k after filtering
    return sorted(results, key=lambda c: c.score, reverse=True)[:top_k]

In [18]:
# Cell 5: query → context → config generator

def build_context_text(retrieved: List[Chunk]) -> str:
    # join all chunk texts
    parts = []
    for c in retrieved:
        parts.append(
            f"---\nSource file: {os.path.basename(c.file_path)} | chunk {c.chunk_index}\n{c.text}"
        )
    return "\n".join(parts)

default_system_prompt = """You are a Cisco IOS XR network engineer.
Using ONLY the information in the CONTEXT, generate a telemetry configuration.
Output valid IOS XR CLI configuration blocks, nothing else.
If you are unsure, make the best reasonable guess but stay consistent with IOS XR syntax.""" 

default_collection_name = "fixed_window_embeddings"

CHAT_MODEL = "gpt-4.1-mini"   # or gpt-4.1, etc.

def generate_telemetry_config(user_query: str, top_k: int = 8, system_prompt =default_system_prompt, collection_name=default_collection_name ) -> str:
    retrieved = retrieve_chunks(user_query, top_k=top_k, collection_name=collection_name)
    context = build_context_text(retrieved)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": (
                f"CONTEXT:\n{context}\n\n"
                f"USER REQUEST:\n{user_query}\n\n"
                "Return only the telemetry model-driven configuration."
            ),
        },
    ]

    resp = client_oa.chat.completions.create(
        model=CHAT_MODEL,
        messages=messages,
        temperature=0.2,
    )

    return resp.choices[0].message.content.strip(), retrieved


In [19]:
# Cell 6: example query

query = (
    "Generate telemetry configuration for Cisco IOS XR about BGP. "
    "Use gRPC with no TLS, telemetry server 192.0.2.0 port 57500. "
    "Choose relevant BGP sensor paths."
)

config, retrieved_chunks = generate_telemetry_config(query, top_k=10)
print(config)


telemetry model-driven
 subscription BGP-STATE
  sensor-group-id 101
  sample-interval 10000
  sensor-path bgp-state
 !
 sensor-group 101
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af/neighbor-entries/neighbor
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af/route-rd-entries/route-rd-entry
 !
 destination-group DG1
  address-family ipv4
  encoding self-describing-gpb
  transport udp
  udp-port 57500
  vrf default
  destination-address 192.0.2.0
  protocol grpc
  no-tls
 !
 subscription BGP-STATE
  sensor-group-id 101
  destination-id DG1
  sample-interval 10000
 exit
exit


In [20]:
retrieved_chunks

[Chunk(id=8, file_path='../data/yang/vendor/cisco/xr/701/openconfig-local-routing.yang', chunk_index=8, text='FILE: openconfig-local-routing.yang\nCHUNK: 8\n;\n              uses local-static-nexthop-config;\n            }\n            container state {\n              config false;\n              description\n                "Operational state parameters relating to the\n                 next-hop entry";\n              uses local-static-nexthop-config;\n              uses local-static-nexthop-state;\n            }\n            uses oc-if:interface-ref;\n          }\n        }\n      }\n    }\n  }\n\n  grouping local-aggreg...', score=0.40097156),
 Chunk(id=4, file_path='../data/yang/vendor/cisco/xr/701/openconfig-local-routing.yang', chunk_index=4, text='FILE: openconfig-local-routing.yang\nCHUNK: 4\nt:ip-address;\n        type local-defined-next-hop;\n      }\n      description\n        "The next-hop that is to be used for the static route\n         - this may be specified as an IP ad

Issues:

subscription is at the top, with sensor-path directly under it → wrong hierarchy.

No sensor-group / destination-group separation.

Uses stream grpc, transport grpc, destination-ip, destination-port, no tls → this smells like a mashup of IOS XE / NX-OS syntax, not XR.

No sample-interval or destination-group-id.

Extra exit, end that you probably don’t want the model to output.

So: semantically okay, CLI invalid for XR.

In [13]:
system_prompt = """
You are a Cisco IOS XR network engineer.

Always output telemetry model-driven configuration for IOS XR 7.x
using EXACTLY this structure (adapt names as needed):

telemetry model-driven
 sensor-group <SENSOR_GROUP_NAME>
  sensor-path <PATH_1>
  sensor-path <PATH_2>
 !
 destination-group DG-GRPC
  address-family ipv4
   destination <DEST_IP>
    port <DEST_PORT>
    encoding self-describing-gpb
    protocol grpc no-tls
 !
 subscription <SUBSCRIPTION_NAME>
  sensor-group-id <SENSOR_GROUP_NAME> sample-interval <INTERVAL_MS>
  destination-group-id DG-GRPC
 !

Rules:
- Do NOT use 'stream', 'transport', 'destination-ip', 'destination-port', or 'no tls' commands.
- Put ALL sensor-path lines inside a sensor-group.
- Put ALL destination settings inside a destination-group as shown.
- Use only IOS XR syntax.
- Use the CONTEXT below only to choose relevant sensor-paths.
- Output only configuration, no explanations.
"""


In [ ]:
# Example query

query = (
    "Generate telemetry configuration for Cisco IOS XR about BGP. "
    "Use gRPC with no TLS, telemetry server 192.0.2.0 port 57500. "
    "Choose relevant BGP sensor paths."
)

config, retrieved = generate_telemetry_config(query, top_k=10, system_prompt= system_prompt)
print(config)


telemetry model-driven
 sensor-group SG-BGP
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af/advertised-paths/advertised-path
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af/paths/path
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af/neighbors/neighbor
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af/rib/ipv4-unicast/attributes
 !
 destination-group DG-GRPC
  address-family ipv4
   destination 192.0.2.0
    port 57500
    encoding self-describing-gpb
    protocol grpc no-tls
 !
 subscription SUB-BGP
  sensor-group-id SG-BGP sample-interval 10000
  destination-group-id DG-GRPC
 !


In [ ]:
config, retrieved = generate_telemetry_config(query, top_k=10, system_prompt= system_prompt, collection_name='catalog_embeddings')
print(config)

telemetry model-driven
 sensor-group BGP-SG
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/afs/af
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/neighbors/neighbor
  sensor-path Cisco-IOS-XR-ipv4-bgp-oper:bgp/instances/instance/instance-active/default-vrf/rib/afi-safis/afi-safi
 !
 destination-group DG-GRPC
  address-family ipv4
   destination 192.0.2.0
    port 57500
    encoding self-describing-gpb
    protocol grpc no-tls
 !
 subscription BGP-Sub
  sensor-group-id BGP-SG sample-interval 10000
  destination-group-id DG-GRPC
 !
